# Schema Group

> Grouping related configuration schemas for better organization

In [ ]:
#| default_exp core.schema_group

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, Any, Optional

In [ ]:
#| export
@dataclass
class SchemaGroup:
    """A group of related configuration schemas.
    
    Use SchemaGroup to organize multiple related schemas under a single
    collapsible section in the settings sidebar. This is useful for
    applications with many configuration options.
    
    Example:
        ```python
        media_group = SchemaGroup(
            name="media",
            title="Media Settings",
            schemas={
                "scanner": MEDIA_SCAN_SCHEMA,
                "player": MEDIA_PLAYER_SCHEMA,
                "library": MEDIA_LIBRARY_SCHEMA
            },
            default_open=True,
            description="Configure media scanning, playback, and library"
        )
        ```
    
    Attributes:
        name: Internal identifier for the group (used in unique IDs)
        title: Display title shown in the sidebar
        schemas: Dictionary mapping schema keys to schema definitions
        icon: Optional icon/SVG element for the group
        default_open: Whether the group is expanded by default
        description: Optional description of the group
    """
    name: str
    title: str
    schemas: Dict[str, Dict[str, Any]]
    icon: Optional[Any] = None
    default_open: bool = True
    description: Optional[str] = None

    def get_schema(self, schema_name: str) -> Optional[Dict[str, Any]]:
        """Get a specific schema from the group by name."""
        return self.schemas.get(schema_name)

    def get_unique_id(self, schema_name: str) -> str:
        """Generate a unique ID for a schema within this group.
        
        Format: {group_name}_{schema_name}
        Example: "media_scanner"
        """
        return f"{self.name}_{schema_name}"

    def has_configured_schemas(
        self,
        config_dir: Path  # Directory where config files are stored
    ) -> bool:  # True if any schema in group has saved config
        """Check if any schemas in this group have saved configurations."""
        for schema_name in self.schemas:
            config_file = Path(config_dir) / f"{self.get_unique_id(schema_name)}.json"
            if config_file.exists():
                return True
        return False

    def get_configured_schemas(
        self,
        config_dir: Path  # Directory where config files are stored
    ) -> list:  # List of schema names that have saved configs
        """Get list of configured schema names in this group."""
        configured = []
        for schema_name in self.schemas:
            config_file = Path(config_dir) / f"{self.get_unique_id(schema_name)}.json"
            if config_file.exists():
                configured.append(schema_name)
        return configured

In [ ]:
# Example: Create a schema group
database_group = SchemaGroup(
    name="database",
    title="Database Settings",
    schemas={
        "connection": {
            "name": "connection",
            "title": "Connection Settings",
            "type": "object",
            "properties": {
                "host": {"type": "string", "title": "Host", "default": "localhost"}
            }
        },
        "performance": {
            "name": "performance",
            "title": "Performance Settings",
            "type": "object",
            "properties": {
                "pool_size": {"type": "integer", "title": "Pool Size", "default": 10}
            }
        }
    },
    default_open=True,
    description="Configure database connection and performance"
)

print(f"Group name: {database_group.name}")
print(f"Group title: {database_group.title}")
print(f"Unique ID for 'connection': {database_group.get_unique_id('connection')}")
print(f"Connection schema: {database_group.get_schema('connection')['title']}")

Group name: database
Group title: Database Settings
Unique ID for 'connection': database_connection
Connection schema: Connection Settings


In [ ]:
# Example: Check for configured schemas
import tempfile
import json


database_group = SchemaGroup(
    name="database",
    title="Database Settings",
    schemas={
        "connection": {
            "name": "connection",
            "title": "Connection Settings",
            "type": "object",
            "properties": {
                "host": {"type": "string", "title": "Host", "default": "localhost"}
            }
        },
        "performance": {
            "name": "performance",
            "title": "Performance Settings",
            "type": "object",
            "properties": {
                "pool_size": {"type": "integer", "title": "Pool Size", "default": 10}
            }
        }
    },
    default_open=True,
    description="Configure database connection and performance"
)

# Create a temporary config directory
with tempfile.TemporaryDirectory() as tmpdir:
    config_dir = Path(tmpdir)
    
    # Initially, no schemas are configured
    print(f"Has configured schemas (before): {database_group.has_configured_schemas(config_dir)}")
    
    # Save a config file for one schema
    config_file = config_dir / "database_connection.json"
    with open(config_file, 'w') as f:
        json.dump({"host": "prod.example.com"}, f)
    
    # Now it should detect configured schemas
    print(f"Has configured schemas (after): {database_group.has_configured_schemas(config_dir)}")
    print(f"Configured schemas: {database_group.get_configured_schemas(config_dir)}")

Has configured schemas (before): False
Has configured schemas (after): True
Configured schemas: ['connection']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()